Problem 1:

In [2]:
SELECT 
    t.Name AS TerritoryName,
CASE 
WHEN EXISTS (
    SELECT 1
     FROM Sales.SalesOrderHeader soh
     JOIN Sales.SalesTerritory st 
     ON soh.TerritoryID = st.TerritoryID
     WHERE st.TerritoryID = t.TerritoryID
     AND YEAR(soh.OrderDate) = 2013
     GROUP BY st.TerritoryID, MONTH(soh.OrderDate)
     HAVING SUM(soh.TotalDue) > 750000
     ) THEN 'No'
     ELSE 'Yes'
     END AS TrainingRequired
FROM 
    Sales.SalesTerritory t
ORDER BY 
    t.Name ASC;

(10 rows affected)

Total execution time: 00:00:00.095

TerritoryName,TrainingRequired
Australia,Yes
Canada,No
Central,Yes
France,No
Germany,Yes
Northeast,Yes
Northwest,No
Southeast,Yes
Southwest,No
United Kingdom,Yes


Problem 2

Part A:

In [43]:
SELECT cr.Name AS Country, sp.Name AS StateProvince, COUNT(soh.SalesOrderID) AS TotalOrders
FROM Sales.SalesOrderHeader soh
INNER JOIN Person.Address a 
ON soh.ShipToAddressID = a.AddressID
INNER JOIN Person.StateProvince sp
ON a.StateProvinceID = sp.StateProvinceID
INNER JOIN Person.CountryRegion cr
ON sp.CountryRegionCode = cr.CountryRegionCode
GROUP BY cr.Name, sp.Name
ORDER BY cr.Name ASC, TotalOrders DESC

(70 rows affected)

Total execution time: 00:00:00.106

Country,StateProvince,TotalOrders
Australia,New South Wales,3009
Australia,Victoria,1710
Australia,Queensland,1461
Australia,South Australia,466
Australia,Tasmania,197
Canada,British Columbia,3472
Canada,Ontario,339
Canada,Quebec,146
Canada,Alberta,90
Canada,Manitoba,12


Part B:

<span style="background-color: transparent; color: rgb(0, 0, 0); font-family: &quot;Times New Roman&quot;, serif; font-size: 12pt; white-space-collapse: preserve;"><br></span>

In [44]:
SELECT cr.Name AS Country, sp.Name AS StateProvince, 
        COUNT(soh.SalesOrderID) AS TotalOrders
FROM Sales.SalesOrderHeader soh
INNER JOIN Person.Address a 
ON soh.ShipToAddressID = a.AddressID
INNER JOIN Person.StateProvince sp
ON a.StateProvinceID = sp.StateProvinceID
INNER JOIN Person.CountryRegion cr
ON sp.CountryRegionCode = cr.CountryRegionCode
GROUP BY cr.Name, sp.Name
HAVING 
    COUNT(soh.SalesOrderID) >= 100
ORDER BY cr.Name ASC, TotalOrders DESC


(26 rows affected)

Total execution time: 00:00:00.126

Country,StateProvince,TotalOrders
Australia,New South Wales,3009
Australia,Victoria,1710
Australia,Queensland,1461
Australia,South Australia,466
Australia,Tasmania,197
Canada,British Columbia,3472
Canada,Ontario,339
Canada,Quebec,146
France,Seine (Paris),575
France,Seine Saint Denis,391


Problem 3

In [77]:
WITH ProductSales AS (
    SELECT 
        p.ProductID,
        p.Name AS ProductName,
        pc.Name AS CategoryName,
        SUM(sod.OrderQty) AS TotalUnitsSold
    FROM Sales.SalesOrderDetail sod
    JOIN Production.Product p 
    ON sod.ProductID = p.ProductID
    JOIN Production.ProductSubcategory psc 
    ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    JOIN Production.ProductCategory pc ON psc.ProductCategoryID = pc.ProductCategoryID
    GROUP BY 
        p.ProductID, p.Name, pc.Name),
RankedSales AS (
    SELECT 
        ps.ProductID,
        ps.ProductName,
        ps.CategoryName,
        ps.TotalUnitsSold,
        RANK() OVER (PARTITION BY ps.CategoryName ORDER BY ps.TotalUnitsSold DESC) AS SalesRank,
        NTILE(5) OVER (PARTITION BY ps.CategoryName ORDER BY ps.TotalUnitsSold DESC) AS SalesQuintile
    FROM 
        ProductSales ps
)
SELECT 
    CategoryName,
    ProductName,
    TotalUnitsSold,
    SalesRank,
    SalesQuintile
FROM 
    RankedSales
ORDER BY 
    CategoryName,
    SalesRank;


(266 rows affected)

Total execution time: 00:00:00.874

CategoryName,ProductName,TotalUnitsSold,SalesRank,SalesQuintile
Accessories,Water Bottle - 30 oz.,6815,1,1
Accessories,"Sport-100 Helmet, Blue",6743,2,1
Accessories,"Sport-100 Helmet, Black",6532,3,1
Accessories,"Sport-100 Helmet, Red",6266,4,1
Accessories,Patch Kit/8 Patches,3865,5,1
Accessories,Bike Wash - Dissolver,3319,6,2
Accessories,Hitch Rack - 4-Bike,3166,7,2
Accessories,Mountain Tire Tube,3095,8,2
Accessories,Hydration Pack - 70 oz.,2761,9,2
Accessories,Road Tire Tube,2376,10,2
